In [1]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

In [3]:
df = pd.read_csv('./data/Features.csv')

In [5]:
df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle,department,...,p_num_of_times,p_reorder_ratio,uxp_times_bought,uxp_reorder_ratio,product_recency,product_frequency,avg_product_score,avg_user_grade,product_add_to_cart_ratio,u_reorder_ratio
0,2539329,1,prior,1,2,8,-1.0,Soda,soft drinks,beverages,...,35791,0.776480,10,0.9,0.0,0.072917,31.779442,0.0,3.721774,0.694915
1,2539329,1,prior,1,2,8,-1.0,Organic Unsweetened Vanilla Almond Milk,soy lactosefree,dairy eggs,...,15935,0.810982,1,0.0,0.0,0.072917,38.094948,0.0,5.792595,0.694915
2,2539329,1,prior,1,2,8,-1.0,Original Beef Jerky,popcorn jerky,snacks,...,6476,0.740735,10,0.9,0.0,0.072917,35.763434,0.0,4.760037,0.694915
3,2539329,1,prior,1,2,8,-1.0,Aged White Cheddar Popcorn,popcorn jerky,snacks,...,2523,0.539041,2,0.5,0.0,0.072917,30.622671,0.0,6.495838,0.694915
4,2539329,1,prior,1,2,8,-1.0,XL Pick-A-Size Paper Towel Rolls,paper goods,household,...,1214,0.441516,2,0.5,0.0,0.072917,21.102965,0.0,3.116969,0.694915


In [7]:
del df['eval_set']

In [9]:
df.columns

Index(['order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'product_name', 'aisle', 'department',
       'product_id', 'add_to_cart_order', 'reordered_x', 'aisle_id',
       'department_id', 'recency', 'frequency', 'reordered_y', 'score',
       'grade', 'u_num_of_orders', 'u_avg_prd', 'p_num_of_times',
       'p_reorder_ratio', 'uxp_times_bought', 'uxp_reorder_ratio',
       'product_recency', 'product_frequency', 'avg_product_score',
       'avg_user_grade', 'product_add_to_cart_ratio', 'u_reorder_ratio'],
      dtype='object')

In [11]:
df2 = df.copy()
df2.drop(columns = ['product_name', 'aisle', 'department', 'order_dow', 'order_hour_of_day', 'product_id', 'aisle_id',
       'department_id'], axis=1, inplace=True)

In [12]:
df2.head()

,order_id,user_id,order_number,days_since_prior_order,add_to_cart_order,reordered_x,recency,frequency,reordered_y,score,...,p_num_of_times,p_reorder_ratio,uxp_times_bought,uxp_reorder_ratio,product_recency,product_frequency,avg_product_score,avg_user_grade,product_add_to_cart_ratio,u_reorder_ratio
0,2539329,1,1,-1.0,1,0,0.0,0.072917,0.012519,3.0,...,35791,0.776480,10,0.9,0.0,0.072917,31.779442,0.0,3.721774,0.694915
1,2539329,1,1,-1.0,2,0,0.0,0.072917,0.012519,3.0,...,15935,0.810982,1,0.0,0.0,0.072917,38.094948,0.0,5.792595,0.694915
2,2539329,1,1,-1.0,3,0,0.0,0.072917,0.012519,3.0,...,6476,0.740735,10,0.9,0.0,0.072917,35.763434,0.0,4.760037,0.694915
3,2539329,1,1,-1.0,4,0,0.0,0.072917,0.012519,3.0,...,2523,0.539041,2,0.5,0.0,0.072917,30.622671,0.0,6.495838,0.694915
4,2539329,1,1,-1.0,5,0,0.0,0.072917,0.012519,3.0,...,1214,0.441516,2,0.5,0.0,0.072917,21.102965,0.0,3.116969,0.694915


In [22]:
df2.columns

Index(['order_id', 'user_id', 'order_number', 'days_since_prior_order',
       'add_to_cart_order', 'reordered_x', 'recency', 'frequency',
       'reordered_y', 'score', 'grade', 'u_num_of_orders', 'u_avg_prd',
       'p_num_of_times', 'p_reorder_ratio', 'uxp_times_bought',
       'uxp_reorder_ratio', 'product_recency', 'product_frequency',
       'avg_product_score', 'avg_user_grade', 'product_add_to_cart_ratio',
       'u_reorder_ratio'],
      dtype='object')

### 재구매 예측 모델링

In [26]:
df2.isnull().sum()

order_id                     0
user_id                      0
order_number                 0
days_since_prior_order       0
add_to_cart_order            0
reordered_x                  0
recency                      0
frequency                    0
reordered_y                  0
score                        0
grade                        0
u_num_of_orders              0
u_avg_prd                    0
p_num_of_times               0
p_reorder_ratio              0
uxp_times_bought             0
uxp_reorder_ratio            0
product_recency              0
product_frequency            0
avg_product_score            0
avg_user_grade               0
product_add_to_cart_ratio    0
u_reorder_ratio              0
dtype: int64

In [48]:
# 50% 샘플링 (랜덤으로 50% 샘플링)
data_sampled = df2.sample(frac=1, random_state=42)

In [50]:
data_sampled.shape

(32434489, 23)

In [15]:
# `reordered`가 0인 데이터와 1인 데이터의 비율을 맞추기 위해 SMOTE 적용
X = df2.drop('reordered_x', axis = 1)
y = df2['reordered_x']

# SMOTE로 오버샘플링 (클래스 1에 대해)
# smote = SMOTE(random_state=42)
# X_resampled, y_resampled = smote.fit_resample(X, y)

# 학습용 데이터와 테스트용 데이터 나누기 (80% 학습, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### 스케일링 하지 않고 LGB 사용

---

In [19]:
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score

pipeline = Pipeline([
    ('classifier', LGBMClassifier(class_weight='balanced', random_state=42, n_jobs=-1))
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

# 예측 결과 평가
print("F1 Score:", f1_score(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("precision_score:", precision_score(y_test, y_pred))
print("recall_score:", recall_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

[LightGBM] [Info] Number of positive: 13392089, number of negative: 9312053
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.173691 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3173
[LightGBM] [Info] Number of data points in the train set: 22704142, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
F1 Score: 0.9093050034217679
Accuracy: 0.8926478161570188
precision_score: 0.905488526611921
recall_score: 0.9131537879764169
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.86      0.87   3995900
           1       0.91      0.91      0.91   5734447

    accuracy                           0.89   9730347
   macro avg       0.89      0.89      0.89   9730347
weighted avg  

### 스케일링 사용후 결과

---

In [23]:
# 모델 파이프라인 구성 (표준화 + LGBMClassifier)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LGBMClassifier(class_weight='balanced', random_state=42, n_jobs=-1))
])

# 모델 학습
pipeline.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 13392089, number of negative: 9312053
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.163029 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3215
[LightGBM] [Info] Number of data points in the train set: 22704142, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 LGBMClassifier(class_weight='balanced', n_jobs=-1,
                                random_state=42))])

In [25]:
# 예측
y_pred = pipeline.predict(X_test)

# 예측 결과 평가
print("F1 Score:", f1_score(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", precision_score(y_test, y_pred))
print("Accuracy:", recall_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

F1 Score: 0.9093060155358005
Accuracy: 0.8926577849690253
F1 Score: 0.9055629058897796
Accuracy: 0.913080197619753
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.86      0.87   3995900
           1       0.91      0.91      0.91   5734447

    accuracy                           0.89   9730347
   macro avg       0.89      0.89      0.89   9730347
weighted avg       0.89      0.89      0.89   9730347



In [21]:
df2['reordered_x'].value_counts()

reordered_x
1    19126536
0    13307953
Name: count, dtype: int64

In [29]:
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline

# 모델 파이프라인 구성 (CatBoostClassifier만 사용)
pipeline = Pipeline([
    ('classifier', CatBoostClassifier(random_state=42, verbose=0))
])

# 모델 학습
pipeline.fit(X_train, y_train)

# 모델 평가
y_pred = pipeline.predict(X_test)

# 예측 결과 평가
print("F1 Score:", f1_score(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("precision score:", precision_score(y_test, y_pred))
print("recall score:", recall_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

F1 Score: 0.9159494557938754
Accuracy: 0.8974937892759631
precision score: 0.8862248372271411
recall score: 0.9477372447596081
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.83      0.87   3995900
           1       0.89      0.95      0.92   5734447

    accuracy                           0.90   9730347
   macro avg       0.90      0.89      0.89   9730347
weighted avg       0.90      0.90      0.90   9730347



In [31]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
import numpy as np


# XGBoost 파이프라인 구성
pipeline = Pipeline([
    ('classifier', XGBClassifier(random_state=42, n_jobs=-1, max_depth=10))
])

# 모델 학습
pipeline.fit(X_train, y_train)

# 모델 평가
y_pred = pipeline.predict(X_test)

# 예측 결과 평가
print("F1 Score:", f1_score(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("precision score:", precision_score(y_test, y_pred))
print("recall score:", recall_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

F1 Score: 0.9160022168840303
Accuracy: 0.8976032406655179
precision score: 0.8866410967274536
recall score: 0.9473745245182317
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.83      0.87   3995900
           1       0.89      0.95      0.92   5734447

    accuracy                           0.90   9730347
   macro avg       0.90      0.89      0.89   9730347
weighted avg       0.90      0.90      0.90   9730347

